In [1]:
#Import our libraries
import time
import requests
import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import sqlalchemy

#Import our helper functions
import helper_functions_labor_distribution

# Step 1: View the list of reports available

In [2]:
#report_path = report_lst[report_lst['ReportName'] == 'Pay Period']['ReportPath'].loc[59]
report_path = helper_functions_labor_distribution.get_report_file_path('Pay Period')

report_path 

"/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/folder[@name='Export to SQL']/report[@name='Pay Period']"

## Step 2: Retrieve the report and export as a CSV

In [3]:
df = helper_functions_labor_distribution.retrieve_report(report_path) 
df 

,monthly_pay_period_number,mqy-end_flag,pay_date,pay_date_sequence,pay_group_code,pay_group,pay_period_id,period_closed,period_end_date,period_start_date,period_type_code,period_type
0,1,None,2013-06-07T00:00:00,1,SALARY,Salaried Biweekly,970,Y,2013-06-01T00:00:00,2013-05-19T00:00:00,R,Reg
1,1,None,2013-06-07T00:00:00,1,HOURLY,Hourly BiWeekly,944,Y,2013-06-01T00:00:00,2013-05-19T00:00:00,R,Reg
2,1,None,2013-06-07T00:00:00,1,DOCTOR,Doctor BiWeekly,996,Y,2013-06-01T00:00:00,2013-05-19T00:00:00,R,Reg
3,1,None,2013-05-24T00:00:00,1,DOCTM,Doctor Monthly,1016,Y,2013-05-31T00:00:00,2013-05-01T00:00:00,R,Reg
4,1,None,2013-05-10T00:00:00,1,SALARY,Salaried Biweekly,968,Y,2013-05-04T00:00:00,2013-04-21T00:00:00,R,Reg
...,...,...,...,...,...,...,...,...,...,...,...,...
2148,4,Q,2021-06-25T00:00:00,2,DOCTOR,Doctor BiWeekly,1993,Y,2021-06-25T00:00:00,2021-06-25T00:00:00,A,Adj
2149,4,Q,2021-09-30T00:00:00,4,DOCTOR,Doctor BiWeekly,2105,Y,2021-09-30T00:00:00,2021-09-30T00:00:00,A,Adj
2150,4,Y,2007-12-31T00:00:00,3,DOCTM,Doctor Monthly,435,Y,2007-12-31T00:00:00,2007-12-31T00:00:00,A,Adj
2151,4,Y,2017-12-30T00:00:00,3,DOCTOR,Doctor BiWeekly,1580,Y,2017-12-30T00:00:00,2017-12-30T00:00:00,A,Adj


In [4]:
df.columns

Index(['monthly_pay_period_number', 'mqy-end_flag', 'pay_date',
       'pay_date_sequence', 'pay_group_code', 'pay_group', 'pay_period_id',
       'period_closed', 'period_end_date', 'period_start_date',
       'period_type_code', 'period_type'],
      dtype='object')

In [5]:
#Remove the null values so that they say 0 instead
df['monthly_pay_period_number'] = df['monthly_pay_period_number'].astype(str).replace("'@xs:nil': 'true'", '0')
df['pay_date_sequence'] = df['pay_date_sequence'].astype(str).replace("'@xs:nil': 'true'", '0')
df['pay_period_id'] = df['pay_period_id'].astype(str).replace("'@xs:nil': 'true'", '0')

# Step 3: Export the report as a CSV

In [6]:
df.to_csv('ukg_bulk_insert.csv', index=False)

# Step 4: Insert the report into BiData2 as a table

In [7]:
helper_functions_labor_distribution.create_sql_table_v2('ZZ_TEST_Payperiod')

# Step 5: Close the API connection

In [8]:
helper_functions_labor_distribution.close_api_connection()

'<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOffResponse</a:Action></s:Header><s:Body><LogOffResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><LogOffResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ServiceId i:nil="true"/><ClientAccessKey>PLAB4</ClientAccessKey><Token i:nil="true"/><Status>LoggedOff</Status><StatusMessage i:nil="true"/><InstanceKey i:nil="true"/></LogOffResult></LogOffResponse></s:Body></s:Envelope>'